In [1]:
import sys, datetime
from gridstatusio import GridStatusClient
import pandas as pd
import os
import numpy as np

startCode = datetime.datetime.now()

os.environ["GRIDSTATUS_API_KEY"] = "API KEY HERE"

client = GridStatusClient()

<b>Renewables Rollout</b>

One way to assess the increase in renewables across the US in real-time is via the fuel mix of respective ISOs. We also have a [Records page](https://www.gridstatus.io/records) and the ability to [set alerts](https://www.gridstatus.io/alerts) if there are specific thresholds of interest.

In [10]:
start = "2019-01-01"
end = "2024-03-25"

def monthly_renewable_growth(df, col1, col2):
    df['year'] = df['interval_start_local'].dt.year
    df['month'] = df['interval_start_local'].dt.month
    
    table = pd.pivot_table(fuelERCOT, values=[col1, col2], index=['year', 'month'],
                       aggfunc="sum", fill_value=0)
    
    return(table)

fuelERCOT = client.get_dataset(
    dataset = 'ercot_fuel_mix',
    start = "2019-01-01", 
    end = "2024-03-25", 
    tz = "US/Central")
ercotTable = monthly_renewable_growth(fuelERCOT, 'solar', 'wind')

fuelSPP = client.get_dataset(
    dataset = 'spp_fuel_mix',
    start = "2019-01-01", 
    end = "2024-03-25", 
    tz = "US/Central")
sppTable = monthly_renewable_growth(fuelSPP, 'solar', 'wind')

fuelCAISO = client.get_dataset(
    dataset = 'caiso_fuel_mix',
    start = "2019-01-01", 
    end = "2024-03-25", 
    tz = "US/Pacific")
caisoTable = monthly_renewable_growth(fuelCAISO, 'solar', 'wind')

Fetching Page 1...Done in 2.43 seconds. 
Fetching Page 2...Done in 2.69 seconds. Total time: 5.13s. Avg per page: 2.56s
Fetching Page 3...Done in 2.52 seconds. Total time: 7.65s. Avg per page: 2.55s
Fetching Page 4...Done in 2.43 seconds. Total time: 10.08s. Avg per page: 2.52s
Fetching Page 5...Done in 2.47 seconds. Total time: 12.55s. Avg per page: 2.51s
Fetching Page 6...Done in 1.39 seconds. Total time: 13.94s. Avg per page: 2.32s

Total number of rows: 550052
Fetching Page 1...Done in 2.95 seconds. 
Fetching Page 2...Done in 3.3 seconds. Total time: 6.25s. Avg per page: 3.12s
Fetching Page 3...Done in 3.16 seconds. Total time: 9.41s. Avg per page: 3.14s
Fetching Page 4...Done in 2.95 seconds. Total time: 12.36s. Avg per page: 3.09s
Fetching Page 5...Done in 3.16 seconds. Total time: 15.52s. Avg per page: 3.1s
Fetching Page 6...Done in 1.53 seconds. Total time: 17.05s. Avg per page: 2.84s

Total number of rows: 548323
Fetching Page 1...Done in 2.57 seconds. 
Fetching Page 2...Done 

In [11]:
ercotTable

solar          wind
year month                            
2019 1      2.863237e+06  7.723380e+07
     2      2.724306e+06  7.407224e+07
     3      3.473812e+06  7.254227e+07
     4      4.915050e+06  8.578077e+07
     5      4.523082e+06  8.574155e+07
...                  ...           ...
2023 11     2.299564e+07  9.366813e+07
     12     2.694672e+07  1.119596e+08
2024 1      2.676962e+07  1.025299e+08
     2      3.173892e+07  1.313347e+08
     3      2.917918e+07  9.051144e+07

[63 rows x 2 columns]

<b>Curtailment</b>

SPP and CAISO examples here as they provide very direct access to this type of data. In other markets there are ways to calculate this that vary from very direct, but delayed (ERCOT, with the 60-Day SCED Generator data) to more indirect (NYISO, wind gen dropping off as constraints start hitting the Central-East interface). The indirect methods are generally applicable across markets, but the specifics (which areas to look at, which transmission lines matter) vary from market to market. 

In [3]:
start = "2023-01-01"
end = "2024-03-25"

curtailCAISO = client.get_dataset(
    dataset = 'caiso_curtailment',
    start = "2023-01-01", 
    end = "2024-03-25", 
    tz = "US/Pacific")

curtailCAISO['year'] = curtailCAISO['interval_start_local'].dt.year
curtailCAISO['month'] = curtailCAISO['interval_start_local'].dt.month

table = pd.pivot_table(curtailCAISO, values=['curtailment_mwh'], index=['year', 'month'],
                       columns = ['curtailment_type', 'curtailment_reason', 'fuel_type'],aggfunc="sum", fill_value=0)
table

Fetching Page 1...Done in 0.3 seconds. 

Total number of rows: 12533


curtailment_mwh                                      
curtailment_type          Economic                       SelfSchCut     
curtailment_reason           Local         System             Local     
fuel_type                    Solar   Wind   Solar   Wind      Solar Wind
year month                                                              
2023 1                       46938   2399    2612   1239          4    2
     2                      175981   9236   17413   1325          0    0
     3                      392559  17963  125125  13106          5    0
     4                      539153  21351  132507   9517        350    0
     5                      214054  13319  198665  22584        279   53
     6                      153081  16772   16835   3309        234    2
     7                       52825   1990    2680    177        187    0
     8                       46015    831    2201     59        162    0
     9                       71395   2376    1573    153        420   91
     10                     197631   6505    6493    280        439   42
     11                      58657   1645   10223    251          0    0
     12                      31119   2811     133     50          0    0
2024 1                       95113   5476     208     34        181    0
     2                      201318  14187   12233   1075         75    0
     3                      503791  36834    4628   1321         20    0

In [4]:
curtailSPP = client.get_dataset(
    dataset = 'spp_ver_curtailments',
    start = "2023-01-01", 
    end = "2024-03-25", 
    tz = "US/Central")

curtailSPP['year'] = curtailSPP['interval_start_local'].dt.year
curtailSPP['month'] = curtailSPP['interval_start_local'].dt.month

table = pd.pivot_table(curtailSPP, values=['wind_redispatch_curtailments','wind_manual_curtailments',
                      'wind_curtailed_for_energy','solar_redispatch_curtailments',
                      'solar_manual_curtailments','solar_curtailed_for_energy'],
                       index=['year', 'month'],
                       aggfunc="sum", fill_value=0)
table

Fetching Page 1...Done in 3.04 seconds. 
Fetching Page 2...Done in 0.64 seconds. Total time: 3.68s. Avg per page: 1.84s

Total number of rows: 129300


solar_curtailed_for_energy  solar_manual_curtailments  \
year month                                                          
2023 1                         6387.80                          0   
     2                         6506.66                          0   
     3                        17154.59                          0   
     4                        20445.15                          0   
     5                          614.66                          0   
     6                          396.01                          0   
     7                            0.00                          0   
     8                            3.21                          0   
     9                          132.39                          0   
     10                        1988.59                          0   
     11                        6658.10                          0   
     12                       11259.83                          0   
2024 1                         2782.87                          0   
     2                        24481.03                          0   
     3                        24143.35                          0   

            solar_redispatch_curtailments  wind_curtailed_for_energy  \
year month                                                             
2023 1                           17774.15                 2375750.64   
     2                           34614.57                 2230922.21   
     3                           65431.65                 3071102.50   
     4                           76843.69                 4029435.24   
     5                           34506.48                 1503737.37   
     6                           13560.76                  519200.95   
     7                            1312.79                  305403.05   
     8                             894.58                  373619.41   
     9                           13653.03                 1937651.99   
     10                          49770.58                 2363963.80   
     11                          34719.87                 2505481.60   
     12                          39855.83                 2598036.03   
2024 1                           25192.42                 1144524.55   
     2                           69494.41                 2707494.49   
     3                           69486.60                 4041790.38   

            wind_manual_curtailments  wind_redispatch_curtailments  
year month                                                          
2023 1                      36748.36                    9053965.40  
     2                     101698.93                   11784127.91  
     3                     153518.79                   14535034.87  
     4                     480086.16                   18545143.14  
     5                      20260.18                    4754681.74  
     6                      29561.65                    2426515.34  
     7                     102186.92                    2437190.97  
     8                     205005.66                    2451357.32  
     9                      47161.31                    8857075.24  
     10                     14334.21                   14559800.78  
     11                     31816.57                   12164602.39  
     12                    163573.06                   10311543.14  
2024 1                     113269.04                    9281395.29  
     2                      15932.47                   12171168.68  
     3                      28332.65                   13838899.53

<b>Battery Deployment</b>

CAISO and ERCOT have by far the most batteries and also the best reporting. Starting with fuel mix you'll see that CAISO's reporting standard is more useful for that type of view, since it measures both ways unlike ERCOT. Switching into their more direct datasets you can see that ERCOT's numbers become more useful. Although these storage-specific datasets have more limited history, they are useful in the going forward sense.

In [5]:
start = "2019-01-01"
end = "2024-03-25"


def monthly_battery_growth(df, col1):
    df['year'] = df['interval_start_local'].dt.year
    df['month'] = df['interval_start_local'].dt.month
    
    table = pd.pivot_table(df, values=[col1], index=['year', 'month'],
                           aggfunc={'min', 'max'}, fill_value=0)
    
    table.columns = [f'{col1}_min', f'{col1}_max']
    table = table.reset_index()
    
    return(table)

fuelERCOT = client.get_dataset(
    dataset = 'ercot_fuel_mix',
    start = "2019-01-01", 
    end = "2024-03-25", 
    tz = "US/Central")

ercotTable = monthly_battery_growth(fuelERCOT, 'power_storage')
ercotTable

Fetching Page 1...Done in 2.32 seconds. 
Fetching Page 2...Done in 2.46 seconds. Total time: 4.78s. Avg per page: 2.39s
Fetching Page 3...Done in 2.49 seconds. Total time: 7.27s. Avg per page: 2.42s
Fetching Page 4...Done in 2.42 seconds. Total time: 9.69s. Avg per page: 2.42s
Fetching Page 5...Done in 2.54 seconds. Total time: 12.23s. Avg per page: 2.45s
Fetching Page 6...Done in 1.5 seconds. Total time: 13.74s. Avg per page: 2.29s

Total number of rows: 550052


,year,month,power_storage_min,power_storage_max
0,2019,1,0.0,0.0
1,2019,2,0.0,0.0
2,2019,3,0.0,0.0
3,2019,4,0.0,0.0
4,2019,5,0.0,0.0
...,...,...,...,...
58,2023,11,989.1,0.0
59,2023,12,877.6,0.0
60,2024,1,1261.1,0.0
61,2024,2,1419.8,0.1


In [6]:
fuelCAISO = client.get_dataset(
    dataset = 'caiso_fuel_mix',
    start = "2019-01-01", 
    end = "2024-03-25", 
    tz = "US/Pacific")
caisoTable = monthly_battery_growth(fuelCAISO, 'batteries')
caisoTable

Fetching Page 1...Done in 2.7 seconds. 
Fetching Page 2...Done in 2.4 seconds. Total time: 5.1s. Avg per page: 2.55s
Fetching Page 3...Done in 2.66 seconds. Total time: 7.76s. Avg per page: 2.59s
Fetching Page 4...Done in 2.6 seconds. Total time: 10.36s. Avg per page: 2.59s
Fetching Page 5...Done in 2.55 seconds. Total time: 12.91s. Avg per page: 2.58s
Fetching Page 6...Done in 1.48 seconds. Total time: 14.39s. Avg per page: 2.4s

Total number of rows: 549974


,year,month,batteries_min,batteries_max
0,2019,1,120,-108
1,2019,2,112,-117
2,2019,3,121,-123
3,2019,4,126,-118
4,2019,5,117,-109
...,...,...,...,...
58,2023,11,4891,-4714
59,2023,12,4237,-4587
60,2024,1,5488,-4754
61,2024,2,5625,-5876


In [7]:
bessERCOT = client.get_dataset(
    dataset = 'ercot_energy_storage_resources',
    start = "2023-01-01", 
    end = "2024-03-25", 
    tz = "US/Central")

bessERCOT['year'] = bessERCOT['time_local'].dt.year
bessERCOT['month'] = bessERCOT['time_local'].dt.month

ercotTable = pd.pivot_table(bessERCOT, values=['total_charging', 'total_discharging',], index=['year', 'month'],
                   aggfunc="sum", fill_value=0)
ercotTable

Fetching Page 1...Done in 0.57 seconds. 

Total number of rows: 31660


total_charging  total_discharging
year month                                   
2023 12       -1045542.354         817297.907
2024 1        -1535688.685        1260556.638
     2        -1687418.803        1395513.250
     3        -1451265.983        1189166.710

In [8]:
bessCAISO = client.get_dataset(
    dataset = 'caiso_storage',
    start = "2023-01-01", 
    end = "2024-03-25", 
    tz = "US/Pacific")
bessCAISO['year'] = bessCAISO['interval_start_local'].dt.year
bessCAISO['month'] = bessCAISO['interval_start_local'].dt.month

caisoTable = pd.pivot_table(bessCAISO, values=['supply', 'stand_alone_batteries', 'hybrid_batteries'],
                        index=['year', 'month'],
                       aggfunc={'min', 'max'}, fill_value=0)
caisoTable

Fetching Page 1...Done in 2.18 seconds. 
Fetching Page 2...Done in 0.3 seconds. Total time: 2.48s. Avg per page: 1.24s

Total number of rows: 105681


hybrid_batteries      stand_alone_batteries       supply      
                        max  min                   max   min    max   min
year month                                                               
2023 3                  411 -382                  2703 -2339   2919 -2405
     4                  426 -428                  3189 -2490   3518 -2828
     5                  478 -444                  3123 -2312   3378 -2570
     6                  440 -443                  3896 -2822   4216 -3081
     7                  439 -406                  3806 -3185   4178 -3338
     8                  458 -439                  4443 -3536   4753 -3879
     9                  480 -391                  4764 -3521   5223 -3788
     10                 431 -415                  4624 -4169   4999 -4446
     11                 490 -459                  4606 -4428   4891 -4714
     12                 655 -518                  3725 -4293   4237 -4587
2024 1                  557 -477                  5004 -4578   5488 -4754
     2                  542 -453                  5330 -5534   5625 -5876
     3                  583 -498                  4947 -5245   5454 -5540